# 歡迎光臨
與舊版本不同，此版本是為了增加彈性而生。<br>
***請注意，使用此檔案前須將所有必要的附屬檔案放到這份Code存在的資料夾中。***
1. 在Vivado中，Generate Bitstream結束後產生的檔案：<br>
    ***(兩個檔名必須改成一樣)***
    * `.hwh` 檔案，位於 `{ project_location }/{project_name}.runs/impl_1/design_1_wrapper.bit`<br>
    * `.bit` 檔案，位於 `{ project_location }/{project_name}.gen/{Design Name 1}/bd/{Design Name 2}/hw_handoff/design_1.hwh`<br><br>
2. 原先Testbench所需的測試檔 (wip)
---
## 連結
點選連結快速跳到你想看的地方
* ### [參數設定](#設定預設值) 
* ### [函數定義](#函數定義) 
* ### [主程式起點](#主程式) 
---

* ### 設定基礎路徑

In [1]:
baseDir = '/home/xilinx/jupyter_notebooks/[TOP] Flexible University Project/'

* ### 匯入所需模組

In [2]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO
from pynq import buffer

import sys
import time
import numpy as np
import psutil

# %load_ext memory_profiler
import gc
import objgraph
from memory_profiler import profile

import my_module

np.set_printoptions(threshold=sys.maxsize)

* #### 匯入C code dll

In [3]:
# # 載入 SO 動態連結檔案
# ifmaps_premodeling_dll = cdll.LoadLibrary(baseDir + 'ifmaps_premodeling.so')

# # 設定 SO 檔案中 sort 函數的參數資料型態
# ifmaps_premodeling_dll.pre_modeling.argtypes = [POINTER(c_uint), c_uint, c_uint, c_uint, c_uint, c_uint]
# ifmaps_premodeling_dll.free_mem.argtypes = [POINTER(c_uint)]
# # 設定 SO 檔案中 sort 函數的傳回值資料型態
# ifmaps_premodeling_dll.pre_modeling.restype = POINTER(c_uint)
# ifmaps_premodeling_dll.free_mem.restype = None


# # 載入 SO 動態連結檔案
# arr_axis_modeling_dll = ctypes.CDLL(baseDir + 'arr_axis_modeling.so')

# # 設定 SO 檔案中 sort 函數的參數資料型態
# arr_axis_modeling_dll.arr_axis_modeling.argtypes = [POINTER(c_uint), c_uint, c_uint, c_uint, c_uint, c_uint]
# arr_axis_modeling_dll.arr_axis_fc_modeling.argtypes = [POINTER(c_uint), c_uint, c_uint, c_uint, c_uint, POINTER(c_float), POINTER(c_float)]
# arr_axis_modeling_dll.findMAX.argtypes = [POINTER(c_float), c_int]
# arr_axis_modeling_dll.free_mem.argtypes = [POINTER(c_uint)]

# # 設定 SO 檔案中 sort 函數的傳回值資料型態
# arr_axis_modeling_dll.arr_axis_modeling.restype = POINTER(c_uint)
# arr_axis_modeling_dll.arr_axis_fc_modeling.restype = POINTER(c_float)
# arr_axis_modeling_dll.findMAX.restype = c_int
# arr_axis_modeling_dll.free_mem.restype = None


In [4]:
# ffi = FFI()
# c_code = """
# unsigned int* arr_axis_modeling(unsigned int arr[],unsigned int ifmaps_height,unsigned int ifmaps_width,unsigned int ifmaps_channel,unsigned int stride, unsigned int weight_height);
# """

# c_lib = ffi.dlopen(baseDir + 'arr_axis_modeling.so')
# ffi.cdef(c_code)

* ### memory test

In [5]:
# def how_many_free_mem():
#     mem = !cat /proc/meminfo | grep 'MemFree'
#     print(mem)

In [6]:
# how_many_free_mem()

* ### OOP-Wire
用法：
* a = wire(name = '', width = 3, data = 1)
* 可以用slice，會回傳boolean array (ex. a[1:3])
* 直接修改Data，如a.data = 12
* 可使用cascade疊加函數組合埠(底下有範例用法)

In [7]:
# import math


# class Wire:
#     def __init__(self, width: int = 32, data: int = 0) -> None:
#         # if len(args) == 1:
#         #     # Check
#         #     for _ in args:
#         #         if not len(_) == 2 and not type(_) == list:
#         #             raise TypeError('請注意輸入格式')
#         # else:
#         #     raise NotImplementedError('不支援同時綁定多條線')
#         if not type(width) == int and not type(data) == int:
#             raise TypeError('請輸入整數')
#         if width < 1 and width % 1 == 0:
#             raise ValueError('寬度須為正整數')
#         self._OverflowChecker(width=width, data=data)
#         self._width = width
#         self._data = data

#     @property
#     def data(self):
#         return self._data

#     @property
#     def width(self):
#         return self._width

#     @data.setter
#     def data(self, newVal):
#         self._OverflowChecker(width=self._width, data=newVal)
#         self._data = newVal

#     def __getitem__(self, idx):
#         # Just access like Verilog
#         l = self._IntToBoolConv(data=self._data, width=self._width)
#         #l_len = len(l)
#         # print('int2bool', l)
#         # print(self)
#         if type(idx) == int:
#             return Wire(data=l[idx], width=1)
#         else:
#             l = l[idx.stop : idx.start + 1: idx.step]
#             # print(l)
#             if len(l) == 0:
#                 raise ValueError('陣列取值Start, Stop的順序錯誤！')
#             # print('bool2int', self._bool2int(l))
#             return Wire(data=self._bool2int(l), width=len(l))

#     # Bitwise Setter
#     def __setitem__(self, idx: int, newVal: bool):
#         data = self._IntToBoolConv()
#         if type(idx) == int:
#             data[idx] = newVal
#         else: 
#             w = len(data[idx.stop:idx.start + 1:idx.step])
#             total_l = len(data)
#             data[idx.stop:idx.start+1:idx.step] = self._IntToBoolConv(width=w, data=newVal)
#             #data[idx] = self._IntToBoolConv(width=len(data[idx]), data=newVal)

#         self._data = self._bool2int(data)

#     def __repr__(self) -> str:
#         bin_arr=self._IntToBoolConv()
#         bin_arr.reverse()

#         return "[{}:0] = {} > {}".format(self._width - 1, self._data, list(map(int, (bin_arr))))

#     def __len__(self):
#         return self._width

#     def _OverflowChecker(self, width: int = None, data: int = None) -> None:
#         if data is None or width is None:
#             data = self._data
#             width = self._width
#         if math.log2(data + 1) > width:
#             raise OverflowError('溢位')

#     def _IntToBoolConv(self, width: int = None, data: int = None) -> list:
#         if data is None or width is None:
#             data = self._data
#             width = self._width
#         arr = ([bool(data & (1 << n)) for n in range(width)])
#         #arr.reverse()
#         return arr

#     def _bool2int(self, bool_array: list) -> int:
#         ret = 0
#         for _ in range(len(bool_array)):
#             ret += int(bool_array[_]) << _

#         return ret

#     def getIntVal(self) -> int:
#         return self._data


# def cascade(*wires) -> Wire:
#     width = 0
#     data = 0
#     for wire in reversed(wires):
#         if not type(wire) == Wire:
#             raise TypeError()
#         # print(wire.data, wire.width)
#         data += wire.data << width
#         width += wire.width

#     return Wire(data=data, width=width)

In [8]:
# # 範例用法
# signal_a = Wire(width=5, data=31)
# signal_b = Wire(width=2, data=3)
# signal_c = Wire(width=3, data=7)
# signal_d = cascade(signal_a, signal_b, signal_c)
# print('PASS 1>', signal_d)

# signal_a = Wire(width=5, data=0b10101)
# signal_b = signal_a[4:2]
# print('PASS 2>', signal_a, ' >> ', signal_b)

# signal_a = Wire(width=5, data=0b11111)
# signal_b = signal_a[2]
# print('PASS 3>', signal_a,  ' >> ', signal_b)

# signal_a[2:0] = 0b0111
# print('PASS 4>', signal_a)

# signal_a = Wire(width=5, data=0b11000)
# print('PASS 5>', signal_a)


* ### 載入Overlay、設定DMA

In [9]:
design_name = baseDir + 'top_module.bit'
ol = Overlay(design_name)
dma = ol.axi_dma_0
dma_send = ol.axi_dma_0.sendchannel
dma_recv = ol.axi_dma_0.recvchannel
mmio = MMIO(0x4000_0000, 0x1000) # <Not Sure Address Setting> allocate (base address, length)

In [10]:
# dma_send.stop()
# dma_recv.stop()
# dma_send.start()
# dma_recv.start()

In [11]:
ol.ip_dict["processing_system7_0"].keys()

dict_keys(['type', 'gpio', 'interrupts', 'parameters', 'driver', 'device'])

In [12]:
# dma.register_map

In [13]:
# dma.buffer_max_size

In [14]:
# dma.description 

In [15]:
max_input_array_size = 2520 #max(send_AXIS.list_size)
pynq_buffer = allocate(shape = (max_input_array_size, ),dtype=np.uint32)
# AXI_class_time = 0

## 設定預設值
---

In [16]:
# MAC_NUM                    = 256
# WEIGHT_BRAM_ADDRESS_WIDTH  = 12
# BIAS_BRAM_ADDRESS_WIDTH    = 9
# BIAS_BRAM_DATA_WIDTH       = 32
# AXIS_PRELOAD_FIFO_DEPTH    = 4
# AXIS_FIFO_SIZE             = 16
# M_AXIS_FIFO_DEPTH          = 4
# C_S_AXIS_TDATA_WIDTH       = 32
# C_S_AXI_DATA_WIDTH         = 32
# C_S_AXI_ADDR_WIDTH         = 5
# C_M_AXIS_TDATA_WIDTH       = 32

# AXI_0 = Wire(width=32, data=0)
# AXI_1 = Wire(width=32, data=0)
# AXI_2 = Wire(width=32, data=0)
# AXI_3 = Wire(width=32, data=0)
# AXI_4 = Wire(width=32, data=0)
# AXI_5 = Wire(width=32, data=0)
# AXI_6 = Wire(width=32, data=0)
# AXI_7 = Wire(width=32, data=0)

# test use int
AXI_INT_0 = 0
AXI_INT_1 = 0
AXI_INT_2 = 0

## 函數定義
---

* ### Read AXI
[API說明](https://pynq.readthedocs.io/en/v3.0.0/pynq_libraries/mmio.html)
[API文件](https://pynq.readthedocs.io/en/v3.0.0/pynq_package/pynq.mmio.html#pynq-mmio)

In [17]:
def read_AXI(addr):
    ret = mmio.read(addr)
#     ret = mmio.read(addr << 2)
#     print('AXI_Received > ', hex(ret), '(h) = ' , ret , '(d)')  
    return ret

* ### Write AXI
[API說明](https://pynq.readthedocs.io/en/v3.0.0/pynq_libraries/mmio.html)
[API文件](https://pynq.readthedocs.io/en/v3.0.0/pynq_package/pynq.mmio.html#pynq-mmio)

In [18]:
def write_AXI(addr, data):
    mmio.write_reg(addr, data)

In [19]:
# # '''
# #     Send AXIS DATA
    
# #     input: 
# #     - data: int (if size = 1) / list (if size != 1)
# #     - size: int, if multiple input
# #     output:
# #     (none)
    
# # '''
# def send_AXIS(data, list_size:int = 1) -> None:
#     assert list_size > 0 # 長度至少要大於0
# #     print('send AXIS start')    

#     input_data = allocate(shape=(list_size,),dtype=np.uint32)
    
#     start = time.time()
    
#     if list_size == 1:
#         if (type(data) == int) or (type(data) == np.int32):
#             input_data[0] = data
# #             print('Send function got 1 data: {:032b}'.format(data))
#         elif type(data) == Wire:
#             input_data[0] = data.data
# #             print('Send function got 1 wire: {}'.format(data))
#     else:
        
#         for _ in range(list_size):
#             input_data[_] = data[_]
            
#     end = time.time()
#     print(end-start)
# #     print(input_data)

    
#     dma_send.transfer(input_data)
#     dma_send.wait()
    
#     del input_data

#     #print('AXIS Data Sent! Data length = {}'.format(list_size))


In [20]:
# '''
#     Send AXIS DATA
    
#     input: 
#     - data: int (if size = 1) / list (if size != 1)
#     - size: int, if multiple input
#     output:
#     (none)
    
# '''
def send_AXIS(data, list_size:int = 1, pynq_buffer = pynq_buffer) -> None:
    if(type(data) == buffer.PynqBuffer):
        dma_send.transfer(data)
        dma_send.wait()
    else:
        assert list_size > 0 # 長度至少要大於0 

        pynq_buffer[:list_size] = data

        dma_send.transfer(pynq_buffer[:list_size])
        dma_send.wait()
        
#     del pynq_buffer

* ### Read State

In [21]:
# def get_bit(data,start,end):
#     data = data>>end
#     data = data&(pow(2,start-end+1)-1)
#     return data

In [22]:
# get_bit(11010,2,0)

In [23]:
# def read_FSM():
#     ret = mmio.read(4 << 2)
#     print('control_load_weight_state      > ', get_bit(ret,4,0))
#     print('control_load_ifmaps_state      > ', get_bit(ret,9,5))
#     print('control_write_weight_state     > ', get_bit(ret,12,10))
#     print('control_write_bias_state       > ', get_bit(ret,15,13))
#     print('control_load_bias_state        > ', get_bit(ret,18,16))
#     print('weight_bram_control_read_state > ', get_bit(ret,20,19))
#     print('weight_bram_control_write_state> ', get_bit(ret,23,21))
#     print('bias_bram_control_read_state   > ', get_bit(ret,25,24))
#     print('bias_bram_control_write_state  > ', get_bit(ret,28,26))
#     return ret

* ### Send Write Weight Start Command

In [24]:
# def write_weight_start(AXI_0:Wire):
#     s = time.time()
#     AXI_0[7:0] = 12
#     e = time.time()
#     global AXI_class_time
#     AXI_class_time += (e-s)
# #     print(e-s)
#     write_AXI(0,AXI_0.data)
#     return AXI_0

In [25]:
# test for int
def write_weight_start(AXI_INT_0):
#     s = time.time()
    AXI_INT_0 = (AXI_INT_0 & 0xFFFFFF00) | 0xC # AXI_0[7:0] = 12
#     e = time.time()
#     global AXI_class_time
#     AXI_class_time += (e-s)
#     print(e-s)
    write_AXI(0,AXI_INT_0)
    return AXI_INT_0

* ### Send Write Bias Command

In [26]:
# def write_bias_start(AXI_0:Wire):
#     s = time.time()
#     AXI_0[7:0] = 34
#     e = time.time()
#     global AXI_class_time
#     AXI_class_time += (e-s)
# #     print(e-s)
#     write_AXI(0,AXI_0.data)
#     return AXI_0

In [27]:
# test for int
def write_bias_start(AXI_INT_0):
#     s = time.time()
    AXI_INT_0 = (AXI_INT_0 & 0xFFFFFF00) | 0x22 # AXI_0[7:0] = 34
#     e = time.time()
#     global AXI_class_time
#     AXI_class_time += (e-s)
#     print(e-s)
    write_AXI(0,AXI_INT_0)
    return AXI_INT_0   

* ### Send Compute Start Command

In [28]:
# def compute_start(AXI_0:Wire):
#     s = time.time()
#     AXI_0[7:0] = 87
#     e = time.time()
#     global AXI_class_time
#     AXI_class_time += (e-s)
# #     print(e-s)
#     write_AXI(0,AXI_0.data)
#     return AXI_0   

In [29]:
# test for int
def compute_start(AXI_INT_0):
#     s = time.time()
    AXI_INT_0 = (AXI_INT_0 & 0xFFFFFF00) | 0x57 # AXI_0[7:0] = 87
#     e = time.time()
#     global AXI_class_time
#     AXI_class_time += (e-s)
#     print(e-s)
    write_AXI(0,AXI_INT_0)
    return AXI_INT_0   

* ### Send Kernel Size

In [30]:
# def set_kernel_size(AXI_2:Wire, kernel_size:int = 3):
#     s = time.time()
#     if 0 < kernel_size < 6:
#         AXI_2[4:0] = 0b00001 << (kernel_size-1)
#     else:
#         AXI_2[4:0] = 0b00100
#     e = time.time()
#     global AXI_class_time
#     AXI_class_time += (e-s)
#     print(e-s)
# #     print(AXI_2[4:0]) # 請務必再三確認這是對的
#     write_AXI(8,AXI_2.data)

#     return AXI_2    

In [31]:
# test for int
def set_kernel_size(AXI_INT_2, kernel_size:int = 3):
#     s = time.time()
    if 0 < kernel_size < 6:
        AXI_INT_2 = (AXI_INT_2 & 0xFFFFFFE0) | (0b00001 << (kernel_size-1))
    else:
        AXI_INT_2 = (AXI_INT_2 & 0xFFFFFFE0) | 0b00100
#     e = time.time()
#     global AXI_class_time
#     AXI_class_time += (e-s)
#     print(e-s)
    write_AXI(8,AXI_INT_2)

    return AXI_INT_2  

* ### Set Ofmaps Channel And Input Channel

In [32]:
# def set_ofmaps_channel_and_input_channel(AXI_0:Wire,ofmaps_channel:int = 0 , input_channel:int = 0):
#     s = time.time()
#     AXI_0[19:8] = input_channel
#     AXI_0[31:20] = ofmaps_channel
#     e = time.time()
#     global AXI_class_time
#     AXI_class_time += (e-s)
#     print(e-s)
#     write_AXI(0,AXI_0.data)
#     return AXI_0

In [33]:
# test for int
def set_ofmaps_channel_and_input_channel(AXI_INT_0,ofmaps_channel:int = 0 , input_channel:int = 0):
#     s = time.time()
    AXI_INT_0 = (AXI_INT_0 & 0xFFF000FF) | (input_channel << 8) # AXI_0[19:8] = input_channel
    AXI_INT_0 = (AXI_INT_0 & 0x000FFFFF) | (ofmaps_channel << 20) # AXI_0[31:20] = ofmaps_channel
#     e = time.time()
#     global AXI_class_time
#     AXI_class_time += (e-s)
#     print(e-s)
    write_AXI(0,AXI_INT_0)
    return AXI_INT_0

* ### Set Stride Function Ofmaps Width

In [34]:
# def set_stride_function_ofmaps_width(AXI_1:Wire,stride:int = 0 , func:int = 0 , ofmaps_width:int = 0):
#     s = time.time()
#     AXI_1.data = 0
    
#     AXI_1[1:0] = func
#     AXI_1[10:2] = ofmaps_width
#     AXI_1[13:11] = stride
#     e = time.time()
#     global AXI_class_time
#     AXI_class_time += (e-s)
#     print(e-s)
    
#     write_AXI(4,AXI_1.data)
#     return AXI_1

In [35]:
# test for int
def set_stride_function_ofmaps_width(AXI_INT_1, stride:int = 0, func:int = 0 , ofmaps_width:int = 0):
#     s = time.time()
    AXI_INT_1 = 0
    AXI_INT_1 = AXI_INT_1 | func | (ofmaps_width << 2) | (stride << 11)
#     AXI_1[1:0] = func
#     AXI_1[10:2] = ofmaps_width
#     AXI_1[13:11] = stride
#     e = time.time()
#     global AXI_class_time
#     AXI_class_time += (e-s)
#     print(e-s)
    
    write_AXI(4,AXI_INT_1)
    return AXI_INT_1

* ### Write Weight

In [36]:
def write_weight(weight_arr: np.ndarray):
    send_AXIS(weight_arr,weight_arr.size)

* ### Write Bias

In [37]:
def write_bias(bias_arr: np.ndarray):
    send_AXIS(bias_arr,bias_arr.size)

* ### Write Ifmaps

In [38]:
def write_ifmaps(ifmaps_arr: np.ndarray):
#     print(ifmaps_arr.size)
#     print(ifmaps_arr)
    
    send_AXIS(ifmaps_arr,ifmaps_arr.size)


* ### First layer ifmaps modeling

In [39]:
# @profile(precision=4,stream=open('memory_profiler.log','w+'))
def first_ifmaps_modeling(file_ifmaps: np.ndarray,weight_width: int,weight_height: int,ifmaps_width: int,ifmaps_height: int,
                          ifmaps_channel: int,stride: int):

    o_channel = (ifmaps_channel+31)//32
    
    ret_np = my_module.pre_modeling(file_ifmaps, ifmaps_height, ifmaps_width, ifmaps_channel, stride, weight_height)
    
    AXIS_arr = my_module.arr_axis_modeling(ret_np,ifmaps_height,ifmaps_width,o_channel,stride,weight_height)
    
    # 將原ret_np的陣列釋放
    my_module.free_mem(ret_np)
    del ret_np

    return AXIS_arr

* ### ofmaps_padding

In [40]:
def ofmaps_padding(ifmaps_in):
    if(ifmaps_in.shape[2]%2==0):
        h_padding = np.zeros((ifmaps_in.shape[0],ifmaps_in.shape[1],1),dtype=np.uint32)
        h_padding[:,1::2].fill(0xffff_ffff)
        ifmaps_in = np.concatenate((h_padding,ifmaps_in), axis=2) 
        h_padding = np.zeros((ifmaps_in.shape[0],ifmaps_in.shape[1],1),dtype=np.uint32)
        h_padding[:,::2].fill(0xffff_ffff)
        ifmaps_in = np.concatenate((ifmaps_in,h_padding), axis=2) 
    else:
        h_padding = np.zeros((ifmaps_in.shape[0],ifmaps_in.shape[1],1),dtype=np.uint32)
        h_padding[:,1::2].fill(0xffff_ffff)
        ifmaps_in = np.concatenate((h_padding,ifmaps_in,h_padding), axis=2) 

    if(ifmaps_in.shape[1]%2==0):
        v_padding = np.zeros((ifmaps_in.shape[0],1,ifmaps_in.shape[2]),dtype=np.uint32)
        v_padding[:,:,::2].fill(0xffff_ffff)
        ifmaps_in = np.concatenate((v_padding,ifmaps_in), axis=1) 
        v_padding = np.concatenate((v_padding[:,:,1:],v_padding[:,:,:1]), axis=2)
        ifmaps_in = np.concatenate((ifmaps_in,v_padding), axis=1) 
    else:
        v_padding = np.zeros((ifmaps_in.shape[0],1,ifmaps_in.shape[2]),dtype=np.uint32)
        v_padding[:,:,::2].fill(0xffff_ffff)
        ifmaps_in = np.concatenate((v_padding,ifmaps_in, v_padding), axis=1)
    return ifmaps_in

* ### ifmaps modeling

In [41]:
def ifmaps_modeling(ifmaps_in,ifmaps_width,ifmaps_height,ifmaps_channel,weight_width,weight_height,stride,padding=True):

    o_channel = (ifmaps_channel+31)//32
    ifmaps_in = ifmaps_in.reshape(ifmaps_height,ifmaps_width,o_channel)
    ifmaps_in = ifmaps_in.transpose(2,0,1)
    
    if padding == True:
        
        ifmaps_in = ofmaps_padding(ifmaps_in)

        o_hight = int(((ifmaps_in.shape[2] - weight_width) / stride + 1))
        o_width = ifmaps_width
        
        ifmaps_width = ifmaps_width+2
        ifmaps_height = ifmaps_height+2
        
    else:
        if (ifmaps_width%stride!=0):
            pad_width = ifmaps_width%stride
            h_padding = np.zeros((ifmaps_in.shape[0],ifmaps_in.shape[1],pad_width),dtype=np.uint32)
            ifmaps_in = np.concatenate((ifmaps_in,h_padding), axis=2) 
            v_padding = np.zeros((ifmaps_in.shape[0],pad_width,ifmaps_in.shape[2]),dtype=np.uint32)
            ifmaps_in = np.concatenate((ifmaps_in,v_padding), axis=1) 
            ifmaps_height = ifmaps_height+pad_width
            ifmaps_width = ifmaps_width+pad_width
        o_hight = ifmaps_height
        o_width = ifmaps_width
        

#     # 取得陣列的指標
#     start = time.time()
    ifmaps_in = ifmaps_in.reshape(-1) # 可以考慮不要再這裡變1維，直接傳入2維到c code裡面會不會比較快?
    out_arr = my_module.arr_axis_modeling(ifmaps_in, ifmaps_height, ifmaps_width, o_channel, stride, weight_height)
#     end = time.time()
#     print('axis_modeling', end-start)
    #將原ret_ptr的陣列釋放

    return out_arr

* ### 驗證AXIS與ANS是否相同

In [42]:
# def validate_with_ans(ans_file_dir):
#     out_arr = []
#     out_arr = np.array(out_arr).astype(int)
    
#     with open(ans_file_dir, 'r') as f:
#         hex_data = f.read()
#     hex_split = hex_data.split()

#     for _ in range(len(hex_split)):
#         out_arr = np.append(out_arr,int(hex_split[_], 16))     
    
#     for _ in range(out_arr.size):
#         if(out_arr[_] != output_data[_]):
#             return 1
#     return 0 

## 主程式起點
---

In [43]:
design_name = baseDir + 'top_module.bit'
ol = Overlay(design_name)
dma = ol.axi_dma_0
dma_send = ol.axi_dma_0.sendchannel
dma_recv = ol.axi_dma_0.recvchannel
mmio = MMIO(0x4000_0000, 0x1000) # <Not Sure Address Setting> allocate (base address, length)

In [44]:
####CIFAR10
# setting_kernel_size    = [3,3,2,3,3,2,3,3,2]
# setting_input_channel  = [3,128,256,256,256,256,256,128,128]
# setting_ofmaps_channel = [128,256,256,256,256,256,128,128,128]
# setting_stride         = [1,1,2,1,1,2,1,1,2]
# setting_function       = [0,0,1,0,0,1,0,0,1]
# setting_ifmaps_width   = [34,32,32,16,16,16,8,8,8]
# setting_ofmaps_width   = [32,32,16,16,16,8,8,8,4]
# setting_out_class      = 10
# setting_use_fcbias     = 1  # 1: Ture  0: False  
# label_name = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# setting_output_array_size = [4096,8192,2048,2048,2048,512,256,256,64]

In [45]:
####CIFAR10
# weight_npy = {}

# weight_npy["weight0_npy"] = np.array(np.load(baseDir + 'weight0.npy'))
# weight_npy["weight1_npy"] = np.array(np.load(baseDir + 'weight1.npy'))
# weight_npy["weight3_npy"] = np.array(np.load(baseDir + 'weight3.npy'))
# weight_npy["weight4_npy"] = np.array(np.load(baseDir + 'weight4.npy'))
# weight_npy["weight6_npy"] = np.array(np.load(baseDir + 'weight6.npy'))
# weight_npy["weight7_npy"] = np.array(np.load(baseDir + 'weight7.npy'))
# weight_npy["fc_weight_npy"] = np.array(np.load(baseDir + 'fc_weight.npy'))

# bias_npy = {}

# bias_npy["bias0_npy"] = np.array(np.load(baseDir + 'bias0.npy'))
# bias_npy["bias1_npy"] = np.array(np.load(baseDir + 'bias1.npy'))
# bias_npy["bias3_npy"] = np.array(np.load(baseDir + 'bias3.npy'))
# bias_npy["bias4_npy"] = np.array(np.load(baseDir + 'bias4.npy'))
# bias_npy["bias6_npy"] = np.array(np.load(baseDir + 'bias6.npy'))
# bias_npy["bias7_npy"] = np.array(np.load(baseDir + 'bias7.npy'))
# bias_npy["fc_bias_npy"] = np.array(np.load(baseDir + 'fc_bias.npy'))


In [46]:
####MNIST
setting_kernel_size    = [3,2,3,2,3,2]
setting_input_channel  = [1,32,32,64,64,128]
setting_ofmaps_channel = [32,32,64,64,128,128]
setting_stride         = [1,2,1,2,1,2]
setting_function       = [0,1,0,1,0,1]
setting_ifmaps_width   = [30,28,14,14,7,7]
setting_ofmaps_width   = [28,14,14,7,7,4]
setting_out_class      = 10
setting_use_fcbias     = 1  # 1: Ture  0: False  
label_name = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

setting_output_array_size = [784,196,392,98,196,64]

In [47]:
####MNIST
weight_npy = {}
weight_buf = {}

weight_npy["weight0_npy"] = np.array(np.load(baseDir + 'weight0.npy'))
weight_npy["weight2_npy"] = np.array(np.load(baseDir + 'weight2.npy'))
weight_npy["weight4_npy"] = np.array(np.load(baseDir + 'weight4.npy'))
weight_npy["fc_weight_npy"] = np.array(np.load(baseDir + 'fc_weight.npy'))

bias_npy = {}
bias_buf = {}

bias_npy["bias0_npy"] = np.array(np.load(baseDir + 'bias0.npy'))
bias_npy["bias2_npy"] = np.array(np.load(baseDir + 'bias2.npy'))
bias_npy["bias4_npy"] = np.array(np.load(baseDir + 'bias4.npy'))
bias_npy["fc_bias_npy"] = np.array(np.load(baseDir + 'fc_bias.npy'))

bias_npy["bias0_npy"] = bias_npy["bias0_npy"].astype(np.uint32)
bias_npy["bias2_npy"] = bias_npy["bias2_npy"].astype(np.uint32)
bias_npy["bias4_npy"] = bias_npy["bias4_npy"].astype(np.uint32)

In [48]:
weight_buf["weight0_buf"] = allocate(shape = (weight_npy["weight0_npy"].size, ),dtype=np.uint32)
weight_buf["weight2_buf"] = allocate(shape = (weight_npy["weight2_npy"].size, ),dtype=np.uint32)
weight_buf["weight4_buf"] = allocate(shape = (weight_npy["weight4_npy"].size, ),dtype=np.uint32)

bias_buf["bias0_buf"] = allocate(shape = (bias_npy["bias0_npy"].size, ),dtype=np.uint32)
bias_buf["bias2_buf"] = allocate(shape = (bias_npy["bias2_npy"].size, ),dtype=np.uint32)
bias_buf["bias4_buf"] = allocate(shape = (bias_npy["bias4_npy"].size, ),dtype=np.uint32)

weight_buf["weight0_buf"][:] = weight_npy["weight0_npy"]
weight_buf["weight2_buf"][:] = weight_npy["weight2_npy"]
weight_buf["weight4_buf"][:] = weight_npy["weight4_npy"]

bias_buf["bias0_buf"][:] = bias_npy["bias0_npy"]
bias_buf["bias2_buf"][:] = bias_npy["bias2_npy"]
bias_buf["bias4_buf"][:] = bias_npy["bias4_npy"]

# del weight_npy
# del bias_npy

In [49]:
output_data = allocate(shape = (max(setting_output_array_size), ),dtype=np.uint32)

* ### Packer

In [50]:
settings = {
    'setting_kernel_size'       : setting_kernel_size,
    'setting_input_channel'     : setting_input_channel,
    'setting_ofmaps_channel'    : setting_ofmaps_channel,
    'setting_stride'            : setting_stride,
    'setting_function'          : setting_function,
    'setting_ifmaps_width'      : setting_ifmaps_width,
    'setting_ofmaps_width'      : setting_ofmaps_width,
    'setting_out_class'         : setting_out_class,
    'setting_use_fcbias'        : setting_use_fcbias,
    'label_name'                : label_name,
    'setting_output_array_size' : setting_output_array_size
}

AXI_wires = [AXI_INT_0, AXI_INT_1, AXI_INT_2]

In [51]:
def main_code(array_from_front_end, AXI_wires=AXI_wires, settings=settings, weight_buf=weight_buf, bias_buf=bias_buf, output_data=output_data):
    
    conv_times = 0 
    pool_times = 0 
    fc_times   = 0
    data_prep_times = 0
    total_start_time = time.time()
    
    # Workround - Data Unpack
    setting_kernel_size = settings['setting_kernel_size']
    setting_input_channel = settings['setting_input_channel']
    setting_ofmaps_channel = settings['setting_ofmaps_channel']
    setting_stride = settings['setting_stride']
    setting_function = settings['setting_function']
    setting_ifmaps_width = settings['setting_ifmaps_width']
    setting_ofmaps_width = settings['setting_ofmaps_width']
    setting_out_class = settings['setting_out_class']
    setting_use_fcbias = settings['setting_use_fcbias']
    label_name = settings['label_name']
    setting_output_array_size = settings['setting_output_array_size']
    
    # Workround - Data Unpack
    AXI_0 = AXI_wires[0]
    AXI_1 = AXI_wires[1]
    AXI_2 = AXI_wires[2]
    
    for now_layer in range(6):
        data_prep_start_time = time.time()

        AXI_0 = set_ofmaps_channel_and_input_channel(AXI_0,setting_ofmaps_channel[now_layer],setting_input_channel[now_layer])
        AXI_1 = set_stride_function_ofmaps_width(AXI_1,setting_stride[now_layer],setting_function[now_layer], setting_ofmaps_width[now_layer])
        AXI_2 = set_kernel_size(AXI_2,setting_kernel_size[now_layer])  
                
        if(setting_function[now_layer] == 0):
            AXI_0 = write_weight_start(AXI_0) 
            dma_send.transfer(weight_buf["weight{}_buf".format(now_layer)])#write_weight(weight_buf["weight{}_buf".format(now_layer)])
            while(read_AXI(12) != 0x1):
                pass

            AXI_0 = write_bias_start(AXI_0)
            dma_send.transfer(bias_buf["bias{}_buf".format(now_layer)])#write_bias(bias_buf["bias{}_buf".format(now_layer)])
            while(read_AXI(12) != 0x2):
                pass
            
        AXI_0 = compute_start(AXI_0)

        if(now_layer == 0):
            ifmaps_arr = first_ifmaps_modeling(array_from_front_end,setting_kernel_size[0],setting_kernel_size[0],
                                               setting_ifmaps_width[0],setting_ifmaps_width[0],setting_input_channel[0],
                                               setting_stride[0])
            my_module.free_mem(array_from_front_end)
            del array_from_front_end
            
        else:
            if(setting_function[now_layer] == 0):
                padding_en = True
            else:
                padding_en = False
            
            ifmaps_arr = ifmaps_modeling(output_data[:setting_output_array_size[now_layer-1]],setting_ifmaps_width[now_layer],
                                         setting_ifmaps_width[now_layer],setting_input_channel[now_layer],
                                         setting_kernel_size[now_layer],setting_kernel_size[now_layer],
                                         setting_stride[now_layer],padding_en)
            
        data_prep_end_time = time.time()
        data_prep_times += data_prep_end_time - data_prep_start_time
        if(setting_function[now_layer] == 0):
            conv_start_time = time.time()
        else:
            pool_start_time = time.time()
            
        dma_recv.transfer(output_data)
        send_AXIS(ifmaps_arr,ifmaps_arr.size)#write_ifmaps(ifmaps_arr)
        dma_recv.wait()   
            
        while(read_AXI(12) != 0xffff_ffff):
            pass
                
        if(setting_function[now_layer] == 0):
            conv_end_time = time.time()
            conv_times += conv_end_time - conv_start_time 
        else:
            pool_end_time = time.time()
            pool_times += pool_end_time - pool_start_time
         
        my_module.free_mem(ifmaps_arr)
        del ifmaps_arr

    # 呼叫 SO 檔案中的 FC 函數
    fc_start_time = time.time()
    answer_arr = my_module.arr_axis_fc_modeling(output_data,
                                                setting_ofmaps_channel[5]*setting_ofmaps_width[5]*setting_ofmaps_width[5],
                                                setting_out_class, setting_ofmaps_width[5],
                                                setting_use_fcbias,
                                                weight_npy["fc_weight_npy"],bias_npy["fc_bias_npy"])
    answer = my_module.findMAX(answer_arr, setting_out_class)
    
    answer_arr_list = answer_arr.tolist()
    my_module.free_float_mem(answer_arr)

    del answer_arr
#     del output_data

    total_end_time = time.time()
    fc_times = total_end_time - fc_start_time
    total_times = (total_end_time-total_start_time)
    
    
    
    return {'result': label_name[answer] ,'probability_arr': answer_arr_list ,'total_time': total_times, 
            'data_prep_time': data_prep_times ,'conv_time':conv_times ,'pool_time':pool_times ,'fc_time':fc_times}

if __name__ == "__main__":
    print(main_code(np.loadtxt(baseDir + 'ifmaps0.txt', dtype=np.uint32)))  # 正常一般就會執行一次Main Code，不用在外面呼叫

{'result': '7', 'probability_arr': [1.2477482380646165e-09, 3.8720452266716165e-07, 2.6563263872958487e-06, 1.2272471394680906e-07, 1.5914446294118534e-07, 4.4833846857272874e-08, 2.8254939620636632e-12, 0.9999939203262329, 1.603923660553619e-09, 2.769385446299566e-06], 'total_time': 0.016180753707885742, 'data_prep_time': 0.007061004638671875, 'conv_time': 0.0049173831939697266, 'pool_time': 0.0034329891204833984, 'fc_time': 0.0005383491516113281}


In [52]:
print("執行1000次前狀況:")
# 獲取系統記憶體使用情況
memory = psutil.virtual_memory()

print("總記憶體:", memory.total)
print("已使用記憶體:", memory.used)
print("可用記憶體:", memory.available)
print("使用率:", memory.percent)

# 獲取進程的記憶體使用情況
process = psutil.Process()  # 默認獲取當前進程的資源使用情況
process_memory = process.memory_info()

print("進程使用的物理記憶體:", process_memory.rss)
print("進程使用的虛擬記憶體:", process_memory.vms)

num_runs = 1000
# 定義函數
# @profile
def main_code_test():
    for _ in range(num_runs):
        main_code(np.loadtxt(baseDir + 'ifmaps0.txt', dtype=np.uint32))


main_code_test()

# gc.collect()
# objgraph.show_most_common_types(limit=50)

print("執行1000次後:")

# 獲取系統記憶體使用情況
memory = psutil.virtual_memory()

print("總記憶體:", memory.total)
print("已使用記憶體:", memory.used)
print("可用記憶體:", memory.available)
print("使用率:", memory.percent)

# 獲取進程的記憶體使用情況
process = psutil.Process()  # 默認獲取當前進程的資源使用情況
process_memory = process.memory_info()

print("進程使用的物理記憶體:", process_memory.rss)
print("進程使用的虛擬記憶體:", process_memory.vms)

執行1000次前狀況:
總記憶體: 517558272
已使用記憶體: 195104768
可用記憶體: 309317632
使用率: 40.2
進程使用的物理記憶體: 84021248
進程使用的虛擬記憶體: 198017024
執行1000次後:
總記憶體: 517558272
已使用記憶體: 195940352
可用記憶體: 308482048
使用率: 40.4
進程使用的物理記憶體: 84279296
進程使用的虛擬記憶體: 198541312


In [53]:
# gc.collect()

# # 獲取系統記憶體使用情況
# memory = psutil.virtual_memory()

# print("總記憶體:", memory.total)
# print("已使用記憶體:", memory.used)
# print("可用記憶體:", memory.available)
# print("使用率:", memory.percent)

# # 獲取進程的記憶體使用情況
# process = psutil.Process()  # 默認獲取當前進程的資源使用情況
# process_memory = process.memory_info()

# print("進程使用的物理記憶體:", process_memory.rss)
# print("進程使用的虛擬記憶體:", process_memory.vms)

In [54]:
'''
FIFO_BURST 0.07627288818359375
FIFO       0.07755110263824463
HP         0.07804184436798095
ORIG       0.0761120843887329
'''
total_times = 0
for x in range(50):
    ret_dic = main_code( np.loadtxt(baseDir + 'ifmaps0.txt', dtype=np.uint32))
    total_time = ret_dic['total_time']
    total_times += total_time
    print(total_time)
print('avg time = ', total_times/50)

0.01668381690979004
0.020182371139526367
0.016091346740722656
0.015717267990112305
0.015770673751831055
0.015274763107299805
0.01534128189086914
0.015293121337890625
0.02037668228149414
0.019341230392456055
0.014981269836425781
0.014603853225708008
0.014545202255249023
0.014834880828857422
0.014760732650756836
0.014560937881469727
0.014665842056274414
0.018670320510864258
0.01790142059326172
0.015138864517211914
0.014595746994018555
0.014566898345947266
0.014671087265014648
0.014511346817016602
0.014595508575439453
0.014620780944824219
0.01987457275390625
0.01821732521057129
0.014986515045166016
0.01459956169128418
0.014487743377685547
0.014792680740356445
0.014806509017944336
0.014661312103271484
0.014593124389648438
0.018171072006225586
0.01851034164428711
0.014985084533691406
0.016091346740722656
0.014821529388427734
0.014658212661743164
0.014750242233276367
0.014649629592895508
0.014799356460571289
0.01860809326171875
0.01546478271484375
0.015007257461547852
0.014640569686889648
0.

## 主程式結束
---

In [55]:
# read_FSM()

In [56]:
# dma.register_map

In [57]:
# print("------------0----------")
# print(AXI_0)
# read_AXI(0)
# print("------------1----------")
# print(AXI_1)
# read_AXI(4)
# print("------------2----------")
# print(AXI_2)
# read_AXI(8)
# print("------------3----------")
# print(AXI_3)
# read_AXI(12)
# print("------------4----------")
# print(AXI_4)
# read_AXI(16)
# print("------------5----------")
# print(AXI_5)
# read_AXI(20)
# print("------------6----------")
# print(AXI_6)
# read_AXI(24)
# print("------------7----------")
# print(AXI_7)
# read_AXI(28)

In [58]:
# print(dma_send.idle)
# print(dma_send.error)
# print(dma_send.running)
# print(dma_recv.idle)
# print(dma_recv.error)
# print(dma_recv.running)

* ### Calling C Function in Python

In [59]:
# import platform
# print(platform.architecture())

In [60]:
# import ctypes
# from ctypes import cdll, c_int32, c_uint, POINTER

# # 載入 SO 動態連結檔案
# mydll = cdll.LoadLibrary('./test.so')

# # 設定 SO 檔案中 sort 函數的參數資料型態
# mydll.array_add_one.argtypes = [POINTER(c_int32), c_uint]

# # 設定 SO 檔案中 sort 函數的傳回值資料型態
# mydll.array_add_one.restype = POINTER(c_int32)

# # 建立 NumPy 資料陣列
# data = np.array([1,2,3,4,5,6,7,8,9])

# # 取得陣列的指標
# dataPtr = data.ctypes.data_as(POINTER(c_int32))

# # 呼叫 SO 檔案中的 sort 函數

# ret_ptr = mydll.array_add_one(dataPtr, data.size)

# # 將回傳指標指定大小及型態  [0]<-不懂用意
# ret_ptr = ctypes.cast(ret_ptr, ctypes.POINTER(ctypes.c_int32*9))[0]

# # 將buffer轉換成numpy array
# ret_arr = np.frombuffer(ret_ptr, np.int32)

# print(ret_arr)
